In [1]:
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langgraph_supervisor import  create_supervisor
from langgraph.prebuilt import  create_react_agent
from openai import AzureOpenAI
import os
import azure.identity
from azure.identity import ClientSecretCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import langgraph
from langgraph.checkpoint.postgres import PostgresSaver
from psycopg import Connection
from psycopg2 import connect
import psycopg
from azure.mgmt.postgresqlflexibleservers import PostgreSQLManagementClient
#import chainlit

In [2]:
try:
    keyVaultName = os.environ["KEY_VAULT_NAME"]
except KeyError:
    # Get input from user if not set
    keyVaultName = input("Please enter your Key Vault name: ")
    # Save for future cells in this session
    os.environ["KEY_VAULT_NAME"] = keyVaultName


keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

azure_openai_endpoint=client.get_secret(name="aoai-endpoint").value
azure_openai_api_key=client.get_secret(name="aoai-api-key").value
azure_openai_api_version = "2024-02-15-preview"

### Implement PostgreSQL Long Term Memory

In [4]:
import urllib.parse
import os

from azure.identity import DefaultAzureCredential

# IMPORTANT! This code is for demonstration purposes only. It's not suitable for use in production. 
# For example, tokens issued by Microsoft Entra ID have a limited lifetime (24 hours by default). 
# In production code, you need to implement a token refresh policy.

def get_connection_uri():

    # Read URI parameters from the environment
    dbhost = client.get_secret(name="postgres-hostname").value 
    dbname = client.get_secret(name="postgres-chatdb").value 
    dbuser = urllib.parse.quote(client.get_secret(name="postgres-dbuser").value)
    sslmode = "require"

    # Use passwordless authentication via DefaultAzureCredential.
    # IMPORTANT! This code is for demonstration purposes only. DefaultAzureCredential() is invoked on every call.
    # In practice, it's better to persist the credential across calls and reuse it so you can take advantage of token
    # caching and minimize round trips to the identity provider. To learn more, see:
    # https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/identity/azure-identity/TOKEN_CACHING.md 
    credential = DefaultAzureCredential()

    # Call get_token() to get a token from Microsft Entra ID and add it as the password in the URI.
    # Note the requested scope parameter in the call to get_token, "https://ossrdbms-aad.database.windows.net/.default".
    password = credential.get_token("https://ossrdbms-aad.database.windows.net/.default").token

    db_uri = f"postgresql://{dbuser}:{password}@{dbhost}/{dbname}?sslmode={sslmode}"
    return db_uri


# Get the connection URI
conn_string = get_connection_uri()

In [5]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)

search_credential =AzureKeyCredential(client.get_secret(name="aisearch-key").value)
search_endpoint =client.get_secret(name="aisearch-endpoint").value
index_name = "csv-glossary-index"


def search_retrieval(user_input:str) -> str:
        """
        Search and retrieve answers from Azure AI Search.
        Returns:
            str
        """
        query = user_input
        search_result = ""
        search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=search_credential)
        vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=2, fields="vector", exhaustive=True)

        r = search_client.search(  
        search_text=query,
        vector_queries=[vector_query],
        select=["id", "notes", "chunk"],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='my-semantic-config',
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=1
    )
        for result in r:  
            print(f"id: {result['id']}")  
            print(f"notes: {result['notes']}")  
            print(f"Score: {result['@search.score']}")  
            print(f"Content: {result['chunk']}")
            search_result += result['chunk']
        return search_result 

In [6]:
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

from psycopg.errors import UndefinedTable

# Check if the checkpoints table exists and call setup if it doesn't
def setup_checkpointer_table(checkpointer):
    try:
        with checkpointer.conn.cursor() as cur:
            # Check if the checkpoints table exists
            cur.execute("""
                SELECT 1
                FROM information_schema.tables
                WHERE table_name = 'checkpoints';
            """)
            if not cur.fetchone():
                print("Table 'checkpoints' does not exist. Setting up...")
                checkpointer.setup()
            else:
                print("Table 'checkpoints' already exists. Skipping setup.")
    except UndefinedTable:
        print("Error: Table 'checkpoints' does not exist. Setting up...")
        checkpointer.setup()

In [ ]:


with PostgresSaver.from_conn_string(conn_string) as checkpointer:
    setup_checkpointer_table(checkpointer)
    model = AzureChatOpenAI(
        model="gpt-4o", 
        api_key=azure_openai_api_key, 
        api_version=azure_openai_api_version, 
        azure_endpoint=azure_openai_endpoint)
    config = {"configurable": {"thread_id": "1", "user_id": "charles.chinny@lg.com"}}
    research_graph = create_react_agent(
    model=model,
    tools=[search_retrieval],
    name="search_expert",
    prompt="You are a world class researcher with access to an azure index. Use only the index response to respond.",
    checkpointer=checkpointer

    
)
    def research_graph_updates(user_input: str):
        inputs = {"messages": [{"role": "user", "content": user_input}]}
        events =  research_graph.stream(input=inputs,config=config, stream_mode="values")
        for event in events:
            print("Assistant:", event["messages"][-1].pretty_print())


    while True:
        try:
            user_input = input("User: ")
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            research_graph_updates(user_input)
        except:
            # fallback if input() is not available
            user_input = "What do you know about LangGraph?"
            print("User: " + user_input)
            research_graph_updates(user_input)
            break
    checkpoint_tuples = list(checkpointer.list(config))

Table 'checkpoints' already exists. Skipping setup.
================================ Human Message =================================

what is bom?
Assistant: None
================================== Ai Message ==================================
Name: search_expert
Tool Calls:
  search_retrieval (call_jpI3Pevvurpdp1lE14RTyYKE)
 Call ID: call_jpI3Pevvurpdp1lE14RTyYKE
  Args:
    user_input: BOM
Assistant: None
id: 99d593c454b5_aHR0cHM6Ly9ibG9ic3RvcmUwNS5ibG9iLmNvcmUud2luZG93cy5uZXQvZXhjZWwtZGF0YS9nbG9zc2FyeV9kYXRhc2V0LmNzdjsx0_chunks_0
notes: Used in web development to connect services.
Score: 0.016393441706895828
Content: Application Programming Interface: a set of rules that allows different software entities to interact.
================================= Tool Message =================================
Name: search_retrieval

Application Programming Interface: a set of rules that allows different software entities to interact.
Assistant: None
================================== Ai Message

In [ ]:
# config1 = {
#     "configurable": {
#         "user_id": "charles.chinny@lg.com",
#         "thread_id": "1"
#     }}
# checkpointer.get_tuples( config=config1)

In [ ]:
checkpoint_tuples

In [ ]:
checkpoint_tuples[0]